<a href="https://colab.research.google.com/github/mskansara/multivariate_lstm/blob/master/Rough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
from numpy import array
import pandas as pd
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import mean_squared_error

dataset = pd.read_csv('SPI.csv')
features = dataset.drop(['SPI'], axis=1)
output = dataset.drop(['Time','Location','Rainfall','Average temp','Evapotranspiration'], axis=1)
print(dataset.head())
#Removing hyphens from dates
features['Time'] = features['Time'].replace('-','',regex=True)

#Assigning numbers to location
#0-Amravati, 1-Akola, 2-Bhandara, 3-Beed, 4-Aurangabad
features['Location'] = features['Location'].replace('Amravati',0,regex=True)
features['Location'] = features['Location'].replace('Akola',1,regex=True)
features['Location'] = features['Location'].replace('Bhandara',2,regex=True)
features['Location'] = features['Location'].replace('Beed',3,regex=True)
features['Location'] = features['Location'].replace('Aurangabad',4,regex=True)

#Removing the headers from features and output
new_header = features.iloc[0] #grab the first row for the header
features = features[1:] #take the data less the header row
features.columns = new_header

new_header = output.iloc[0] #grab the first row for the header
output = output[1:] #take the data less the header row
output.columns = new_header

#Splitting features and output
X_train, X_test, Y_train, Y_test = train_test_split(features, output, test_size=0.33)

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

#Create a model
model = Sequential()
model.add(LSTM(128,activation='relu', input_shape=(1,5)))
model.add(Dense(1, activation='relu'))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

# fit model
model.fit(X_train,Y_train,epochs=50)

trainPredict = model.predict(X_train)
testPredict = model.predict(X_test)

#calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(Y_train[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(Y_test[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

         Time  Location  Rainfall  Average temp  Evapotranspiration   SPI
0  01-01-1901  Amravati      36.8        21.233                3.71  0.00
1  01-02-1901  Amravati      39.9        22.375                4.53  0.66
2  01-03-1901  Amravati      30.9        27.656                6.00  0.00
3  01-04-1901  Amravati      26.1        31.745                7.11  0.95
4  01-05-1901  Amravati       7.3        34.502                7.57 -0.12
Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_25 (LSTM)               (None, 128)               68608     
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 129       
Total params: 68,737
Trainable params: 68,737
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
401/401 [==============================] - 2s 5ms/step -